# Overview

In [1]:
import sys

assert sys.version_info > (3, 7)

sys.version_info

sys.version_info(major=3, minor=7, micro=1, releaselevel='final', serial=0)

In [2]:
from dataclasses import dataclass

@dataclass
class InventoryItem:
    """Class for keeping track of an item in inventory."""
    name: str
    unit_price: float
    quantity_on_hand: int = 0

    def total_cost(self) -> float:
        return self.unit_price * self.quantity_on_hand

In [3]:
try:
    item = InventoryItem()
except TypeError as e:
    print(repr(e))

TypeError("__init__() missing 2 required positional arguments: 'name' and 'unit_price'")


In [4]:
item = InventoryItem("Widget", 1.99)

item

InventoryItem(name='Widget', unit_price=1.99, quantity_on_hand=0)

In [5]:
item.name

'Widget'

In [6]:
item.unit_price = 2.99

item

InventoryItem(name='Widget', unit_price=2.99, quantity_on_hand=0)

# Type Annotations

In [7]:
assert sys.version_info > (3, 6)

In [8]:
def greet(name: str):
    print("Hello, " + name)
    
greet("Jace")

Hello, Jace


In [9]:
## Argument Annotations

In [10]:
greet

<function __main__.greet(name: str)>

In [11]:
greet.__annotations__

{'name': str}

In [12]:
import typing  # stdlib

hints = typing.get_type_hints(greet)

hints  # `dict` with real classes

{'name': str}

In [13]:
import inspect  # stdlib

signature = inspect.signature(greet)  

signature  # `Signature` object

<Signature (name: str)>

In [14]:
signature.parameters

mappingproxy({'name': <Parameter "name: str">})

In [15]:
signature.parameters['name']

<Parameter "name: str">

In [16]:
# KEYWORD_ONLY: greet(name)
# POSITIONAL_ONLY: greet(*, name)
# VAR_POSITIONAL: greet(*names)
# VAR_KEYWORD: greet(**names)

signature.parameters['name'].kind

<_ParameterKind.POSITIONAL_OR_KEYWORD: 1>

In [17]:
signature.parameters['name'].annotation

str

### Return Annotations

In [18]:
from decimal import Decimal

def add_tax(subtotal, rate=0.06) -> Decimal:
    cents = Decimal('0.01')
    return Decimal(subtotal * (1 + rate)).quantize(cents)

add_tax(4.99)

Decimal('5.29')

In [19]:
inspect.signature(add_tax).return_annotation

decimal.Decimal

### Variable Annotations

In [20]:
class Person:
    name: str

In [21]:
Person.__annotations__

{'name': str}

### Circular Annotations

In [22]:
class Node:
    
    def connect_edge(edge: 'Edge'):
        pass
    

class Edge:
    def connect_node(node: Node):
        pass

In [23]:
from __future__ import annotations  


class Node:
    
    def connect_edge(edge: Edge):
        pass
     

class Edge:
    def connect_node(node: Node):
        pass

# Type Checking (with mypy)

In [24]:
%system pip install mypy==0.670

['Requirement already satisfied: mypy==0.670 in ./.venv/lib/python3.7/site-packages (0.670)',
 'Requirement already satisfied: mypy-extensions<0.5.0,>=0.4.0 in ./.venv/lib/python3.7/site-packages (from mypy==0.670) (0.4.1)',
 'Requirement already satisfied: typed-ast<1.4.0,>=1.3.1 in ./.venv/lib/python3.7/site-packages (from mypy==0.670) (1.3.1)']

In [25]:
from mypy import api

def mypy(*filenames):
    """Emulate `$ mypy <filename>` for notebooks."""
    message, _, _ = api.run(filenames)
    print(message or "(no errors)")

In [26]:
%%writefile greet.py

def greet(name: str):
    print("Hello, " + name)
    
greet("Jace")

Overwriting greet.py


In [27]:
mypy('greet.py')

(no errors)


In [28]:
%%writefile greet2.py

def greet(name: str):
    print("Hello, " + name)
    
greet(42)

Writing greet2.py


In [29]:
mypy('greet2.py')

greet2.py:5: error: Argument 1 to "greet" has incompatible type "int"; expected "str"



In [30]:
%%writefile people.py

from typing import Iterable, List


class Person:
    
    def __init__(self, name):
        self.name = name

        
def get_people(*names: Iterable[str]) -> List[Person] :
    return [Person(name) for name in names]
    
    
people = get_people("Alice", "Bob")

people[1].age

Overwriting people.py


In [31]:
mypy('people.py')

people.py:17: error: "Person" has no attribute "age"



# Dataclasses

In [32]:
from dataclasses import dataclass

@dataclass
class InventoryItem:
    """Class for keeping track of an item in inventory."""
    name: str
    unit_price: float
    quantity_on_hand: int = 0

    def total_cost(self) -> float:
        return self.unit_price * self.quantity_on_hand

### `__init__`

In [33]:
InventoryItem("Widget A", 1.99)

InventoryItem(name='Widget A', unit_price=1.99, quantity_on_hand=0)

In [34]:
InventoryItem("Widge B", 1.99, 300)

InventoryItem(name='Widge B', unit_price=1.99, quantity_on_hand=300)

In [35]:
InventoryItem("Widget C", 1.99, quantity_on_hand=400)

InventoryItem(name='Widget C', unit_price=1.99, quantity_on_hand=400)

In [36]:
InventoryItem(name="Widget D", unit_price=1.99, quantity_on_hand=500)

InventoryItem(name='Widget D', unit_price=1.99, quantity_on_hand=500)

In [37]:
try:
    InventoryItem(name="Widget E")
except TypeError as e:
    print(repr(e))

TypeError("__init__() missing 1 required positional argument: 'unit_price'")


### `__repr__`

In [38]:
item = InventoryItem("Widget", 1.99)

repr(item)

"InventoryItem(name='Widget', unit_price=1.99, quantity_on_hand=0)"

In [39]:
eval("InventoryItem(name='Widget', unit_price=1.99, quantity_on_hand=0)")

InventoryItem(name='Widget', unit_price=1.99, quantity_on_hand=0)

### `__eq__`

In [40]:
item_a = InventoryItem("Widget A", 1.99)
item_b = InventoryItem("Widget B", 1.99)
item_x = InventoryItem("Widget A", 1.99, quantity_on_hand=0)

In [41]:
item_a == item_b

False

In [42]:
item_a == item_x

True

### Ordered Dataclasses

In [43]:
@dataclass(order=True)
class Person:
    last_name: str
    first_name: str
        
    def __str__(self):
        return f'{self.first_name} {self.last_name}'

In [44]:
people = [
    Person(first_name="Alice", last_name="Smith"),
    Person(first_name="Bob", last_name="Smith"),
    Person(first_name="Carl", last_name="Davidson"),
]

for person in people:
    print(person)

Alice Smith
Bob Smith
Carl Davidson


In [45]:
people.sort()

In [46]:
for person in people:
    print(person)

Carl Davidson
Alice Smith
Bob Smith


### Frozen Dataclasses

In [47]:
@dataclass(frozen=True)
class Badge:
    number: int
        
badges = [Badge(1001), Badge(1002), Badge(1003)]

In [48]:
try:
    badges[1].number = 1004
except AttributeError as e:
    print(repr(e))

FrozenInstanceError("cannot assign to field 'number'")


### Field Customization 

In [49]:
from dataclasses import field

@dataclass(order=True)
class Person:
    name: str = field(compare=False)
    age: int
    
    def __str__(self):
        return f'{self.name} ({self.age})'

In [50]:
people = [
    Person("Alice Smith", 30),
    Person("Bob Smith", 25),
    Person("Carl Davidson", 41),
]

for person in people:
    print(person)

Alice Smith (30)
Bob Smith (25)
Carl Davidson (41)


In [51]:
people.sort()

In [52]:
for person in people:
    print(person)

Bob Smith (25)
Alice Smith (30)
Carl Davidson (41)


# Serialization (with datafiles)

In [53]:
%system pip install datafiles==0.2b5

['Requirement already satisfied: datafiles==0.2b5 in ./.venv/lib/python3.7/site-packages (0.2b5)',
 'Requirement already satisfied: ruamel.yaml<0.16.0,>=0.15.46 in ./.venv/lib/python3.7/site-packages (from datafiles==0.2b5) (0.15.88)',
 'Requirement already satisfied: tomlkit<0.6.0,>=0.5.3 in ./.venv/lib/python3.7/site-packages (from datafiles==0.2b5) (0.5.3)',
 'Requirement already satisfied: classproperties<0.2.0,>=0.1.3 in ./.venv/lib/python3.7/site-packages (from datafiles==0.2b5) (0.1.3)',
 'Requirement already satisfied: minilog<2.0.0,>=1.2.3 in ./.venv/lib/python3.7/site-packages (from datafiles==0.2b5) (1.2.3)',
 'Requirement already satisfied: cached_property<2.0,>=1.5 in ./.venv/lib/python3.7/site-packages (from datafiles==0.2b5) (1.5.1)']